### 야놀자 크롤링

In [14]:
#진행순서
# 1. location ,name, level, score, review_count, link 에 대한 spider.py 모듈 생성
# 2. price, room_type 은 selenium 으로 가져오기 -> why? 쿠키정보를 headers 값으로 사용해야하는 문제때문에 scrapy 불가능

In [15]:
import scrapy, requests
from scrapy.http import TextResponse
import pandas as pd
import json

### 1.room_type, price 제외 한 컬럼 spider.py

In [16]:
# 1. 프로젝트 생성

In [6]:
!scrapy startproject yanolja_hotel

New Scrapy project 'yanolja_hotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/yanolja_hotel

You can start your first spider with:
    cd yanolja_hotel
    scrapy genspider example example.com


In [8]:
!tree yanolja_hotel

yanolja_hotel
├── scrapy.cfg
├── yanolja.csv
├── yanolja_hotel
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── items.cpython-36.pyc
│   │   └── settings.cpython-36.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-36.pyc
│       │   └── spider.cpython-36.pyc
│       └── spider.py
└── yanolja_hotel.csv

4 directories, 15 files


In [ ]:
# 2. xpath 찾기 : headers 설정

In [8]:
#link

In [5]:
#호텔 key값 가져오기
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36", 
       "X-Requested-With": "XMLHttpRequest"
          }
checkin_date = "2021-03-04"
checkout_date = "2021-03-05"
region = "서울"
num=0
page = 1
key_info = []
while True:
    parent_url = f"https://www.yanolja.com/api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0\
&checkinDate={checkin_date}&checkoutDate={checkout_date}&hotel=1&keyword=서울&lat=37.50681&lng=127.06624&page={page}&limit=20&\
searchKeyword={region}&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes="
    req = requests.get(parent_url, headers=headers)
    response = req.json()
    #print('page=', str(page))
    page += 1
    
    if response["motels"]["counts"] > 0:
        key = response["motels"]
        key_info.append(key)
    else:
        break
hotel_keys = []
for i in key_info:
    for j in i['lists']:
        hotel_keys.append(j['key'])        
len(hotel_keys)

415

In [7]:
#랑크 모으기
links=[]
for hotel_key in hotel_keys:
    url = "https://www.yanolja.com/hotel/" + hotel_key
    links.append(url)
len(links), links[0]

(415, 'https://www.yanolja.com/hotel/3013410')

In [8]:
# 상세 페이지
link = links[0]
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36"}
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.yanolja.com/hotel/3013410>

In [9]:
try:
    location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
    location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
    name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
    level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
    score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
    review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()   
except:
    location = ["페이지 없음"]
    location = ["페이지 없음"]
    name = ["페이지 없음"]
    level = ["페이지 없음"]
    score = ["페이지 없음"]
    review_count = ["페이지 없음"]

location ,name, level, score, review_count

('서울특별시 영등포구 경인로 870', '페어필드 바이 메리어트 서울', '4성급', '4.7', '8,695')

In [10]:
locations, names, levels, scores, review_counts, loading_urls = [], [], [], [], [], []

num1 = 0
for link in links[:3]:
    try:
        headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36"}
        req = requests.get(link, headers=headers)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
    except:
        location = "페이지 없음"
        location = "페이지 없음"
        name = "페이지 없음"
        level = "페이지 없음"
        score = "페이지 없음"
        review_count = "페이지 없음"
    
    locations.append(location)
    names.append(name)
    levels.append(level)
    scores.append(score)
    review_counts.append(review_count)
    loading_urls.append(link)
    num1+=1
    print(str(num1) ,name)

1 페어필드 바이 메리어트 서울
2 나인트리 프리미어 명동2
3 영등포 GMS 호텔


In [11]:
len(locations), len(names), len(levels), len(scores), len(review_counts), len(loading_urls)

(3, 3, 3, 3, 3, 3)

In [13]:
df = pd.DataFrame({
    "Name" : names, "Level":levels, "Score":scores, "Reviews":review_counts, "Location":locations,"Link":loading_urls
})
df

,Name,Level,Score,Reviews,Location,Link
0,페어필드 바이 메리어트 서울,4성급,4.7,"8,695",서울특별시 영등포구 경인로 870,https://www.yanolja.com/hotel/3013410
1,나인트리 프리미어 명동2,4성급,4.8,"2,141",서울특별시 중구 마른내로 28,https://www.yanolja.com/hotel/3009497
2,영등포 GMS 호텔,3성급,4.7,"5,552",서울특별시 영등포구 경인로 867,https://www.yanolja.com/hotel/3008321


In [393]:
# 3. items.py : location ,name, level, score, review_count, link

In [1]:
%%writefile yanolja_hotel/yanolja_hotel/items.py
import scrapy

class YanoljaHotelItem(scrapy.Item):
    location = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    link = scrapy.Field()

Overwriting yanolja_hotel/yanolja_hotel/items.py


In [3]:
# 4. spider.py

In [ ]:
%%writefile yanolja_hotel/yanolja_hotel/spiders/spider.py
import scrapy
from yanolja_hotel.items import YanoljaHotelItem
import json



In [402]:
# 5. settings.py : robots.txt = False로 수정여부 파악

In [403]:
# 6. 프로젝트 실행

In [ ]:
# 7. argument 설정

In [ ]:
# 8. pipeline.py : 크롤링한 데이터를 mongodb에 저장

In [ ]:
# 9. scrapy 프로젝트에 pymongo 모듈 생성